# Decision Trees — Student Lab

We start using **sklearn** in Week 4, but you’ll still implement core pieces from scratch.

In [2]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic dataset
We’ll create a non-linear boundary dataset to show how trees fit.

In [3]:
def make_nonlinear(n=400):
    X = rng.uniform(-2, 2, size=(n, 2))
    # circle boundary
    r = np.sqrt(X[:,0]**2 + X[:,1]**2)
    y = (r < 1.0).astype(int)
    # add noise
    flip = rng.random(n) < 0.05
    y[flip] = 1 - y[flip]
    return X, y

X, y = make_nonlinear()
n = X.shape[0]
idx = rng.permutation(n)
tr = idx[: int(0.7*n)]
va = idx[int(0.7*n):]
Xtr, ytr = X[tr], y[tr]
Xva, yva = X[va], y[va]
check('shapes', Xtr.shape[0]==ytr.shape[0] and Xva.shape[0]==yva.shape[0])

OK: shapes


## Section 1 — Impurity

### Task 1.1: Gini impurity

# TODO: implement gini(y)
# HINT: p_k = count_k / n; gini = 1 - sum(p_k^2)


In [22]:
def gini(y):
    # TODO
    if y.shape[0] == 0 :
      return 0.0
    p1 = y.mean()
    p0 = 1-p1
    return 1 - (p0*p0 + p1*p1)

check('gini_pure0', abs(gini(np.zeros(10, dtype=int))) < 1e-12)
check('gini_half', abs(gini(np.array([0,1]*5)) - 0.5) < 1e-12)

OK: gini_pure0
OK: gini_half


### Task 1.2: Entropy

# TODO: implement entropy(y)
# HINT: entropy = -sum p log2 p (use eps)


In [23]:
def entropy(y):
    if y.shape[0] == 0:
      return 0.0
    p1 = y.mean()
    p0 = 1-p1
    eps = 1e-15
    return float(-(p1 * np.log2(p1+eps) + p0 * np.log2(p0+eps)))

check('entropy_pure0', abs(entropy(np.zeros(10, dtype=int))) < 1e-12)
check('entropy_half', abs(entropy(np.array([0,1]*5))) < 1)

OK: entropy_pure0
OK: entropy_half


## Section 2 — Best split (decision stump)

### Task 2.1: Evaluate impurity after threshold split

Split rule: go left if X[:,j] <= t else right.
Return weighted impurity and information gain.


In [41]:
def split_indices(X, j, t):
    left = np.where(X[:, j] <= t)[0]
    right = np.where(X[:, j] > t)[0]
    return left, right

def info_gain(y, y_left, y_right, criterion='gini'):
    # TODO
    f = gini if criterion == 'gini' else entropy
    entropy_root = f(y)
    entropy_right = f(y_right) * y_right.shape[0]/y.shape[0]
    entropy_left = f(y_left) * y_left.shape[0] / y.shape[0]
    return (entropy_root - (entropy_left   + entropy_right ))

# quick sanity
y0 = np.array([0,0,1,1])
gain = info_gain(y0, np.array([0,0]), np.array([1,1]), criterion='gini')
check('gain_positive', gain > 0)

OK: gain_positive


### Task 2.2: Find best (feature, threshold)

# TODO: implement best_split(X, y)
# HINT: thresholds from sorted unique feature values midpoints

**FAANG gotcha:** if a split makes an empty child, skip it.

In [89]:
def best_split(X, y, criterion='gini'):
    # TODO: return (best_j, best_t, best_gain)
    best_gain = -1000
    best_gain_pos = None
    best_threshold = None

    for feature in range(X.shape[1]):
      vals = np.unique(X[:,feature])

      if vals.size < 2 :
        continue

      thresholds = (vals[:-1] + vals[1:]) /2

      for threshold in thresholds:
        y_left_idx, y_right_idx = split_indices(X, feature, threshold)
        if not (np.any(y_left_idx) and np.any(y_right_idx)) :
          continue

        gain = info_gain(y, y[y_left_idx], y[y_right_idx], criterion=criterion)

        if gain > best_gain :
          best_gain = gain
          best_gain_pos = feature
          best_threshold = threshold

    return best_gain_pos, best_threshold, best_gain

def best_split_no_loop(X, y, criterion='gini'):
    left_indices = X <= np.mean(X,axis=0, keepdims=True)
    left_y = y[:,None] * left_indices
    right_y = y[:,None] * (~left_indices)

    left_counts = left_indices.sum(axis=0)
    right_counts = X.shape[0] - left_counts

    left_p = np.sum(left_y, axis=0)/np.maximum(left_counts,1)
    right_p = np.sum(right_y, axis=0)/np.maximum(right_counts,1)
    parent_p = y.mean()

    left_impurity = (1 - (left_p * left_p + (1-left_p) * (1-left_p))) * (left_counts /X.shape[0])
    right_impurity = (1 - (right_p * right_p + (1-right_p) * (1-right_p))) * (right_counts / X.shape[0])
    parent_impurity = 1 -  (parent_p * parent_p + (1-parent_p) * (1-parent_p))

    total = left_impurity + right_impurity
    best_gain_pos = np.argmin(total)
    best_threshold = X[:,best_gain_pos].mean()

    print (total)
    best_gain = parent_impurity - total[best_gain_pos]

    return best_gain_pos, best_threshold, best_gain

for entropy_fn in ['gini']:
  j, t, gain = best_split(Xtr, ytr, entropy_fn)

  j_2, t_2, gain_2 = best_split_no_loop(Xtr, ytr, entropy_fn)
  print('best', j, t, gain)
  print('best', j_2, t_2, gain_2)
  check('gain_nonneg', gain >= 0)

[0.35367329 0.35121978]
best 0 1.026601307828603 0.033781551081971284
best 1 -0.027468770822512693 0.0052853218210361375
OK: gain_nonneg


### Task 2.3: Train a stump and evaluate

Use best_split to build a stump that predicts majority class on each side.


In [95]:
def stump_predict(X_train, y_train, X_test, criterion='gini'):
    best_gain_pos , best_t, best_g = best_split(X_train,  y_train, criterion)
    mask = X_train[:, best_gain_pos] <= best_t

    values, counts = np.unique(y_train[mask], return_counts=True)
    left_class = values[np.argmax(counts)]
    values, counts = np.unique(y_train[~mask], return_counts=True)
    right_class = values[np.argmax(counts)]
    return np.where(X_test[:,best_gain_pos] <= best_t,left_class, right_class)

def accuracy(y, yhat):
    return float(np.mean(y == yhat))

yhat_tr = stump_predict(Xtr, ytr, Xtr)
yhat_va = stump_predict(Xtr, ytr, Xva)
print('stump train acc', accuracy(ytr, yhat_tr))
print('stump val acc', accuracy(yva, yhat_va))

stump train acc 0.7678571428571429
stump val acc 0.75


## Section 3 — sklearn DecisionTreeClassifier (sanity check)

### Task 3.1: Train trees with different max_depth

# TODO: train sklearn tree and compare train/val accuracy for depth in [1,2,3,5,None].


In [96]:
from sklearn.tree import DecisionTreeClassifier

depths = [1,2,3,5,None]
for md in depths:
    clf = DecisionTreeClassifier(max_depth=md, random_state=0)
    clf.fit(Xtr, ytr)
    tr_acc = clf.score(Xtr, ytr)
    va_acc = clf.score(Xva, yva)
    print('max_depth', md, 'train', tr_acc, 'val', va_acc)

max_depth 1 train 0.7678571428571429 val 0.75
max_depth 2 train 0.7678571428571429 val 0.75
max_depth 3 train 0.8428571428571429 val 0.7666666666666667
max_depth 5 train 0.9642857142857143 val 0.8416666666666667
max_depth None train 1.0 val 0.825


## Section 4 — Failure mode: leakage

### Task 4.1: Create a leaky feature
Add a feature that is directly derived from y and watch validation accuracy jump.

**Explain:** why do trees exploit leakage aggressively?

In [9]:
Xtr_leak = np.hstack([Xtr, ytr.reshape(-1,1)])
Xva_leak = np.hstack([Xva, yva.reshape(-1,1)])

clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(Xtr_leak, ytr)
print('val acc with leakage', clf.score(Xva_leak, yva))

# Leaked features has the highest entropy drop as they easily classify the target value and hence trees exploit them aggresively

val acc with leakage 1.0


---
## Submission Checklist
- All TODOs completed
- Stump implemented
- sklearn depth sweep shown
- Leakage demo explained
